In [2]:
import copy
import cProfile
import glob
import gzip
import json
import os 
import shutil
import sys
import time
import warnings
import zipfile
from datetime import date

import cdsapi
import h5py
import numpy
import numpy as np
import pandas as pd
import urllib3
import xarray as xr
import ydata_profiling

sys.path.append(os.getcwd() + "/../cds-backend/code/")
import cds_eua4 as eua

warnings.filterwarnings("ignore")

sys.path.insert(0, os.getcwd() + "/../resort/rasotools-master/")
import matplotlib
import matplotlib.font_manager as font_manager
import matplotlib.pylab as plt
import matplotlib.pyplot as maplt
import seaborn

import rasotools

matplotlib.rcParams.update({"font.size": 20})
matplotlib.rcParams["figure.figsize"] = (20, 10)
font = {
    "family": "normal",
    # 'weight' : 'bold',
    "size": 22,
}
matplotlib.rc("font", **font)

no config found


In [58]:
with open("/users/staff/uvoggenberger/scratch/hug/config_v11/active.json") as json_file:
    active_v11 = json.load(json_file)
sids = list(active_v11.keys())

In [60]:
[s for s in sids if "59431" in s]

['0-20000-0-59431', '0-20300-0-59431']

In [40]:
def seconds_to_datetime(seconds, ref="1900-01-01"):
    """from seconds to datetime64"""
    seconds = np.asarray(seconds)
    return pd.to_datetime(seconds, unit="s", origin=ref, errors="coerce")

In [ ]:
22.780000686645508, 108.55000305175781,

In [41]:
seconds_to_datetime(2492982000)

Timestamp('1978-12-31 23:00:00')

In [4]:
import rs_drift
from rs_drift import drift

In [5]:
rs_drift.drift.trajectory()

TypeError: trajectory() missing 6 required positional arguments: 'lat', 'lon', 'u', 'v', 'pressure', and 'temperature'

In [6]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]


def datetime_to_seconds(dates, ref="1900-01-01T00:00:00"):
    """from datetime64 to seconds since 1900-01-01 00:00:00"""
    return ((dates - np.datetime64(ref)) / np.timedelta64(1, "s")).astype(np.int64)


def seconds_to_datetime(seconds, ref="1900-01-01"):
    """from seconds to datetime64"""
    seconds = np.asarray(seconds)
    return pd.to_datetime(seconds, unit="s", origin=ref, errors="coerce")

In [3]:
with open("/users/staff/uvoggenberger/scratch/hug/config_v9/active.json") as json_file:
    active_v9 = json.load(json_file)
with open("/users/staff/uvoggenberger/scratch/hug/config_v11/active.json") as json_file:
    active_v11 = json.load(json_file)

In [4]:
import h5py
import glob
import numpy
import numpy as np
import pandas as pd

def datetime_to_seconds(dates, ref="1900-01-01T00:00:00"):
    """from datetime64 to seconds since 1900-01-01 00:00:00"""
    return ((dates - np.datetime64(ref)) / np.timedelta64(1, "s")).astype(np.int64)

def seconds_to_datetime(seconds, ref="1900-01-01"):
    """from seconds to datetime64"""
    seconds = np.asarray(seconds)
    return pd.to_datetime(seconds, unit="s", origin=ref, errors="coerce")



input_str = "0401820161013230000"
date = (
    str(input_str[5:9]) + "-" + str(input_str[9:11]) + "-" + str(input_str[11:13])
)  # + ' ' + str(input_str[13:15]) + ':' + str(input_str[15:17])

""" New Files will go online in a few days. """
file_dir = glob.glob('/mnt/users/scratch/leo/scratch/converted_v11/long/*'+str(input_str[:5])+'*_CEUAS_merged_v1.nc')[0]
""" For now use the old data: """
# file_dir = glob.glob("/mnt/users/scratch/leo/scratch/converted_v9/*" + str(input_str[:5]) + "*_CEUAS_merged_v1.nc")[0]

# create output dict
output = {}
# open file
with h5py.File(
    file_dir,
    "r",
) as file:
    # indexing
    dt_from = datetime_to_seconds(np.datetime64(date))
    dt_to = dt_from + 48 * 60 * 60
    rts = file["recordindices"]["recordtimestamp"][:]
    idx = np.where(np.logical_and((rts >= dt_from), (rts <= dt_to)))[0]
    idx_d = file["recordindices"]["126"][idx]

    #reading data
    output["temperature"] = file["observations_table"]["observation_value"][idx_d[0] : idx_d[-1]]
    output["pressure"] = file["observations_table"]["z_coordinate"][idx_d[0] : idx_d[-1]]
    output["date_time"] = seconds_to_datetime(
        file["observations_table"]["date_time"][idx_d[0] : idx_d[-1]]
    )
    output["fg_depar"] = file["era5fb"]["fg_depar@body"][idx_d[0] : idx_d[-1]]
    output["an_depar"] = file["era5fb"]["an_depar@body"][idx_d[0] : idx_d[-1]]
    
    output["latitude_displacement"] = file["advanced_homogenisation"]["latitude_displacement"][idx_d[0] : idx_d[-1]]
    output["longitude_displacement"] = file["advanced_homogenisation"]["longitude_displacement"][idx_d[0] : idx_d[-1]]
    output["RAOBCORE_bias_estimate"] = file["advanced_homogenisation"]["RAOBCORE_bias_estimate"][idx_d[0] : idx_d[-1]]
    output["RASE_bias_estimate"] = file["advanced_homogenisation"]["RASE_bias_estimate"][idx_d[0] : idx_d[-1]]
    output["RICH_bias_estimate"] = file["advanced_homogenisation"]["RICH_bias_estimate"][idx_d[0] : idx_d[-1]]
    output["RISE_bias_estimate"] = file["advanced_homogenisation"]["RISE_bias_estimate"][idx_d[0] : idx_d[-1]]

# put output to file
df = pd.DataFrame.from_dict(output)
# select only the chosen hour -> take care, sometimes it's not a fixed time, it may update with each ascent level
# skip to investigate the data manually 
df = df[df.date_time.dt.hour == int(input_str[13:15])]
display(df)
# df.to_csv("test_file_"+str(input_str)+".csv")

,temperature,pressure,date_time,fg_depar,an_depar,latitude_displacement,longitude_displacement,RAOBCORE_bias_estimate,RASE_bias_estimate,RICH_bias_estimate,RISE_bias_estimate
169,275.600006,86800.0,2016-10-13 23:00:00,0.701121,0.291900,0.042170,-0.044194,0.296366,NaN,0.296366,NaN
170,279.000000,92500.0,2016-10-13 23:00:00,-0.025871,-0.218117,0.023124,-0.028220,0.195295,NaN,0.195295,NaN
171,283.399994,100000.0,2016-10-13 23:00:00,0.669929,0.547652,0.001225,0.000273,0.066160,0.06616,0.066160,0.06616
172,283.399994,100600.0,2016-10-13 23:00:00,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN


In [5]:
import h5py
import glob
import numpy
import numpy as np
import pandas as pd

def datetime_to_seconds(dates, ref="1900-01-01T00:00:00"):
    """from datetime64 to seconds since 1900-01-01 00:00:00"""
    return ((dates - np.datetime64(ref)) / np.timedelta64(1, "s")).astype(np.int64)

def seconds_to_datetime(seconds, ref="1900-01-01"):
    """from seconds to datetime64"""
    seconds = np.asarray(seconds)
    return pd.to_datetime(seconds, unit="s", origin=ref, errors="coerce")



input_str = "0401820161013230000"
date = (
    str(input_str[5:9]) + "-" + str(input_str[9:11]) + "-" + str(input_str[11:13])
)  # + ' ' + str(input_str[13:15]) + ':' + str(input_str[15:17])

""" New Files will go online in a few days. """
# file_dir = glob.glob('/mnt/users/scratch/leo/scratch/converted_v11/long/*'+str(input_str[:5])+'*_CEUAS_merged_v1.nc')[0]
""" For now use the old data: """
file_dir = glob.glob("/mnt/users/scratch/leo/scratch/converted_v9/*" + str(input_str[:5]) + "*_CEUAS_merged_v1.nc")[0]

# create output dict
output = {}
# open file
with h5py.File(
    file_dir,
    "r",
) as file:
    # indexing
    dt_from = datetime_to_seconds(np.datetime64(date))
    dt_to = dt_from + 48 * 60 * 60
    rts = file["recordindices"]["recordtimestamp"][:]
    idx = np.where(np.logical_and((rts >= dt_from), (rts <= dt_to)))[0]
    idx_d = file["recordindices"]["126"][idx]

    #reading data
    output["temperature"] = file["observations_table"]["observation_value"][idx_d[0] : idx_d[-1]]
    output["pressure"] = file["observations_table"]["z_coordinate"][idx_d[0] : idx_d[-1]]
    output["date_time"] = seconds_to_datetime(
        file["observations_table"]["date_time"][idx_d[0] : idx_d[-1]]
    )
    output["fg_depar"] = file["era5fb"]["fg_depar@body"][idx_d[0] : idx_d[-1]]
    output["an_depar"] = file["era5fb"]["an_depar@body"][idx_d[0] : idx_d[-1]]
    
    output["latitude_displacement"] = file["advanced_homogenisation"]["latitude_displacement"][idx_d[0] : idx_d[-1]]
    output["longitude_displacement"] = file["advanced_homogenisation"]["longitude_displacement"][idx_d[0] : idx_d[-1]]
    output["RAOBCORE_bias_estimate"] = file["advanced_homogenisation"]["RAOBCORE_bias_estimate"][idx_d[0] : idx_d[-1]]
    output["RASE_bias_estimate"] = file["advanced_homogenisation"]["RASE_bias_estimate"][idx_d[0] : idx_d[-1]]
    output["RICH_bias_estimate"] = file["advanced_homogenisation"]["RICH_bias_estimate"][idx_d[0] : idx_d[-1]]
    output["RISE_bias_estimate"] = file["advanced_homogenisation"]["RISE_bias_estimate"][idx_d[0] : idx_d[-1]]

# put output to file
df = pd.DataFrame.from_dict(output)
# select only the chosen hour -> take care, sometimes it's not a fixed time, it may update with each ascent level
# skip to investigate the data manually 
df = df[df.date_time.dt.hour == int(input_str[13:15])]
display(df)
# df.to_csv("test_file_"+str(input_str)+".csv")

,temperature,pressure,date_time,fg_depar,an_depar,latitude_displacement,longitude_displacement,RAOBCORE_bias_estimate,RASE_bias_estimate,RICH_bias_estimate,RISE_bias_estimate
165,275.600006,86800.0,2016-10-13 23:00:00,0.701121,0.291900,NaN,NaN,NaN,NaN,NaN,NaN
166,279.000000,92500.0,2016-10-13 23:00:00,-0.025871,-0.218117,NaN,NaN,-0.0,-0.0,-0.097980,-0.097980
167,283.399994,100000.0,2016-10-13 23:00:00,0.669929,0.547652,NaN,NaN,-0.0,-0.0,-6.315786,-6.315786


In [33]:
int(input_str[13:15])


23

## CDS API
### Backend

In [ ]:
"""
    {
        "day": ["1"],
        "format": "csv",
        "month": "7",
        "statid": "11035",
        "pressure_level": ["100"],  # USE hPa for CDSAPI! Pa everywhere else!
        "observed_variable": ["air_temperature"],
        "year": "1997",
        "optional": ["obs_minus_bg", "obs_minus_an", "bias_estimate"],
    },
    """

['temperature', 'air_temperature',
 'u_component_of_wind', 'eastward_wind_speed',
 'v_component_of_wind', 'northward_wind_speed',
 'wind_speed',
 'wind_direction', 'wind_from_direction',
 'relative_humidity',
 'specific_humidity',
 'dew_point_temperature',
 'geopotential', 'geopotential_height',
 'dew_point_depression', 'dewpoint_departure','dewpoint_depression', 'dew_point_departure']

In [22]:
import cdsapi

c = cdsapi.Client(
    url="https://cds-test.copernicus-climate.eu/api/v2",
    key="2548:a32dce56-b04a-42fc-8fc3-a972f94772ad",
    progress=True,
    retry_max=5,
)
r = c.retrieve(
    "insitu-comprehensive-upper-air-observation-network",
    {
        # "statid": "58666",
        "observed_variable": ["air_temperature", "relative_humidity", 'eastward_wind_speed', 'northward_wind_speed', 'geopotential'],
        "format": "csv",
        "date": ["19820201", "19820231"],
        "area": [48,13,45.8,17.8],
        # 45.8°N bis 48°N und 13°E bis 17.8°E.
        # "optional": [
        #     "RISE_bias_estimate",
        #     "RICH_bias_estimate",
        #     "RASE_bias_estimate",
        #     "RAOBCORE_bias_estimate",
        #     "latitude_displacement",
        #     "longitude_displacement",
        #     "obs_minus_bg",
        #     "obs_minus_an",
        # ],
        "dummy": "test1548741",
    },
    target="download4.zip",
)

In [18]:
tempdir = "./tmp_heimo/"
delete_dl = True
with zipfile.ZipFile("download4.zip", "r") as zip_ref:
    try:
        os.mkdir(tempdir)
    except:
        pass

    zip_ref.extractall(tempdir)

for i in glob.glob(tempdir + "*"):
    print(i)
    df = pd.read_csv(i, header=14)
    display(df)

#if delete_dl:
#    shutil.rmtree(tempdir)

./tmp_heimo/CDS_CUON_output_file.csv


,date_time,latitude,longitude,report_id,station_id,z_coordinate,variable,observation_value
0,1982-02-28 23:00:00+00,47.270000,16.629999,100000000051793,0-20000-0-12812,10000.0,air_temperature,228.300003
1,1982-02-28 23:00:00+00,47.270000,16.629999,100000000051793,0-20000-0-12812,12900.0,air_temperature,229.500000
2,1982-02-28 23:00:00+00,47.270000,16.629999,100000000051793,0-20000-0-12812,15000.0,air_temperature,230.100006
3,1982-02-28 23:00:00+00,47.270000,16.629999,100000000051793,0-20000-0-12812,17500.0,air_temperature,230.699997
4,1982-02-28 23:00:00+00,47.270000,16.629999,100000000051793,0-20000-0-12812,20000.0,air_temperature,221.500000
...,...,...,...,...,...,...,...,...
21751,1982-02-28 12:00:00+00,46.029999,13.180000,100000000037908,0-20001-0-16045,50000.0,relative_humidity,0.582795
21752,1982-02-28 12:00:00+00,46.029999,13.180000,100000000037908,0-20001-0-16045,70000.0,relative_humidity,0.209468
21753,1982-02-28 12:00:00+00,46.029999,13.180000,100000000037908,0-20001-0-16045,85000.0,relative_humidity,0.208628
21754,1982-02-28 12:00:00+00,46.029999,13.180000,100000000037908,0-20001-0-16045,100000.0,relative_humidity,0.452413


In [19]:
df.station_id.unique()

array(['0-20000-0-12812', '0-20000-0-14240', '0-20000-0-16040',
       '0-20001-0-16045'], dtype=object)

## EUA Request
### Backend

### local machine

In [9]:
del df_v11

In [4]:
# input_str = '0401820161013230000'
# file_dir = glob.glob('/mnt/users/scratch/leo/scratch/converted_v11/long/*'+str(input_str[:5])+'*_CEUAS_merged_v1.nc')[0]
# date = str(input_str[5:9]) + '-' + str(input_str[9:11]) + '-' + str(input_str[11:13]) + ' ' + str(input_str[13:15]) + ':' + str(input_str[15:17])

rq = {
    "statid": "33791",
    "date": ["19800601"],
    "variable": ["relative_humidity"],
    "format": "nc",
    # "cdm": ["station_configuration/platform_type"],
    # "cdm": ["source_configuration/source_file"],
    # "dummy": "14854ad8add478asdopf48a4s48d45",
}
df_v11 = eua.vm_request_wrapper(rq, overwrite=True)# , vm_url="http://127.0.0.1:8007")


In [6]:
df_v11.to_dataframe()

,hur,lat,lon,obs,plev,report_id,station_id,time
0,0.486059,47.93,33.330002,0,25000.0,100000000001081,/mnt/scratch/sc,1980-06-01 00:00:00
1,0.459986,47.93,33.330002,0,30000.0,100000000001081,/mnt/scratch/sc,1980-06-01 00:00:00
2,0.568539,47.93,33.330002,0,40000.0,100000000001081,/mnt/scratch/sc,1980-06-01 00:00:00
3,0.469204,47.93,33.330002,0,50000.0,100000000001081,/mnt/scratch/sc,1980-06-01 00:00:00
4,0.469838,47.93,33.330002,0,50200.0,100000000001081,/mnt/scratch/sc,1980-06-01 00:00:00
5,0.477378,47.93,33.330002,0,55100.0,100000000001081,/mnt/scratch/sc,1980-06-01 00:00:00
6,0.616930,47.93,33.330002,0,57300.0,100000000001081,/mnt/scratch/sc,1980-06-01 00:00:00
7,0.606766,47.93,33.330002,0,70000.0,100000000001081,/mnt/scratch/sc,1980-06-01 00:00:00
8,0.506946,47.93,33.330002,0,79000.0,100000000001081,/mnt/scratch/sc,1980-06-01 00:00:00
9,0.386177,47.93,33.330002,0,81700.0,100000000001081,/mnt/scratch/sc,1980-06-01 00:00:00


In [5]:
for i in df_v11.keys():
    print(i)
    display(df_v11[i].station_configuration.platform_type[:])


dest_0-20000-0-33791_air_temperature


array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

dest_20999-coordinate-orphan_era5.conv.??????.33791.txt.gz_air_temperature


array([2, 2, 2, ..., 2, 2, 2], dtype=int32)

In [10]:
df_v11.to_dataframe()

AttributeError: 'CDMDatasetList' object has no attribute 'to_dataframe'

In [6]:
print(len(glob.glob("./test_dest/*relative_humidity.nc")))
print(len(glob.glob("./test_dest/*specific_humidity.nc")))
print(len(glob.glob("./test_dest/*dew_point_temperature.nc")))
print(len(glob.glob("./test_dest/*air_temperature.nc")))


825
825
825
865


In [31]:
df[df.time.dt.month == 5].time.unique()

array(['1982-05-01T06:00:00.000000000', '1982-05-02T06:00:00.000000000',
       '1982-05-03T06:00:00.000000000', '1982-05-04T06:00:00.000000000',
       '1982-05-05T06:00:00.000000000', '1982-05-06T06:00:00.000000000',
       '1982-05-07T06:00:00.000000000', '1982-05-08T06:00:00.000000000',
       '1982-05-09T06:00:00.000000000', '1982-05-10T06:00:00.000000000',
       '1982-05-11T06:00:00.000000000', '1982-05-12T06:00:00.000000000',
       '1982-05-13T06:00:00.000000000', '1982-05-14T06:00:00.000000000',
       '1982-05-15T06:00:00.000000000', '1982-05-16T06:00:00.000000000',
       '1982-05-17T06:00:00.000000000', '1982-05-18T06:00:00.000000000',
       '1982-05-19T06:00:00.000000000', '1982-05-20T06:00:00.000000000',
       '1982-05-21T06:00:00.000000000', '1982-05-22T06:00:00.000000000',
       '1982-05-23T06:00:00.000000000', '1982-05-24T06:00:00.000000000',
       '1982-05-25T06:00:00.000000000', '1982-05-26T06:00:00.000000000',
       '1982-05-27T06:00:00.000000000', '1982-05-28

In [22]:
for i in range(1980, 2024):
    print(i, len(df[df.time.dt.year == i]))

1980 21
1981 0
1982 7758
1983 0
1984 0
1985 0
1986 3622
1987 6418
1988 8145
1989 7649
1990 7811
1991 7849
1992 7677
1993 8374
1994 8756
1995 12320
1996 12378
1997 12072
1998 11730
1999 12308
2000 21619
2001 21838
2002 18553
2003 15216
2004 13915
2005 13039
2006 14068
2007 13938
2008 14100
2009 13776
2010 13823
2011 14011
2012 13632
2013 13595
2014 20765
2015 25892
2016 27825
2017 619163
2018 1256949
2019 1312221



KeyboardInterrupt



### remote VM machine

In [ ]:
rq = {
    "statid": ["06610"],
    "variable": ["air_temperature"],
    "date": "20001231-20010101",
    "format": "nc",
    "pressure_level": ["85000"],
}
df_v11 = eua.vm_request_wrapper(rq, overwrite=True)  # , vm_url="http://127.0.0.1:8009")
df = df_v11.to_dataframe()
df

### multi file output

In [17]:
rq = {
    # "statid": ["72520"],
    "area": [55, 10, 45, 25],
    "variable": ["air_temperature"],#["northward_wind_speed", "eastward_wind_speed"],
    # "optional": ["wind_bias_estimate"],
    "date": "20000101-20000101",
    "format": "nc",
}
df_v11 = eua.vm_request_wrapper(rq)  # , overwrite=True, vm_url='http://127.0.0.1:8009')
for i in df_v11:
    stat = i.split("_air_temperature")[0].split("dest_")[-1]
    stat = glob.glob('/mnt/users/scratch/leo/scratch/converted_v11/long/*'+stat+'*v1.nc')[0]
    print(i.split("_air_temperature")[0].split("dest_")[-1])
    !scp {stat} sis@136.156.154.104:/data/private/v1.10_test/bbox/


0-20000-0-10184
0-20000-0-10184_CEUAS_merged_v1.nc            100% 4966MB  20.5MB/s   04:01    
0-20000-0-10488
0-20000-0-10488_CEUAS_merged_v1.nc            100%  275MB  17.3MB/s   00:15    
0-20000-0-10771
0-20000-0-10771_CEUAS_merged_v1.nc            100% 8326MB  16.7MB/s   08:19    
0-20000-0-10868
0-20000-0-10868_CEUAS_merged_v1.nc            100% 4873MB  12.8MB/s   06:21    
0-20000-0-11010
0-20000-0-11010_CEUAS_merged_v1.nc            100% 1357MB  18.4MB/s   01:13    
0-20000-0-11120
0-20000-0-11120_CEUAS_merged_v1.nc            100% 1540MB  24.0MB/s   01:04    
0-20000-0-11240
0-20000-0-11240_CEUAS_merged_v1.nc            100% 1378MB  23.0MB/s   00:59    
0-20000-0-11520
0-20000-0-11520_CEUAS_merged_v1.nc            100% 5760MB  15.0MB/s   06:24    
0-20000-0-11723
0-20000-0-11723_CEUAS_merged_v1.nc            100%   39MB  50.3MB/s   00:00    
0-20000-0-11952
0-20000-0-11952_CEUAS_merged_v1.nc            100% 5299MB  17.3MB/s   05:05    
0-20000-0-12120
0-20000-0-12120_CEUAS_me

## EUA CDM
### File Access

In [6]:
gfile = glob.glob('/users/staff/uvoggenberger/scratch/converted_v12/0-20001-0-53845_CEUAS_merged_v1.nc')[0]

with eua.CDMDataset(
    gfile
) as file:
    display(file)
    # display(file["header_table"]['report_id'][:])
    display(file["source_configuration"])
    display(file["observations_table"])
    rn = file["station_configuration"]["record_number"][:]
    display(rn)
len(np.unique(rn)) == len(rn)

File: <HDF5 file "0-20001-0-53845_CEUAS_merged_v1.nc" (mode r)>
Filesize: 533.71 MB
Filename: /users/staff/uvoggenberger/scratch/converted_v12/0-20001-0-53845_CEUAS_merged_v1.nc
(G)roups/(V)ariables: 

 - G | advanced_homogenisation______________________ : : 11
 - G | advanced_uncertainty_________________________ : : 9
 - G | crs__________________________________________ : : 5
 - G | era5fb_______________________________________ : : 72
 - G | header_table_________________________________ : : 56
 - G | observations_table___________________________ : : 51
 - G | observed_variable____________________________ : : 10
 - G | recordindices________________________________ : : 13
 - G | sensor_configuration_________________________ : : 13
 - G | source_configuration_________________________ : : 3
 - G | station_configuration________________________ : : 48
 - G | station_configuration_codes__________________ : : 8
 - G | station_type_________________________________ : : 5
 - G | units___________

source_configuration:

index_____________________________________________ : : (49589,)
source_file_______________________________________ : : (49589, 70)
string70__________________________________________ : : (70,)

observations_table:

adjustment_id_____________________________________ : : (9246959,)
advanced_assimilation_feedback____________________ : : (9246959,)
advanced_homogenisation___________________________ : : (9246959,)
advanced_qc_______________________________________ : : (9246959,)
advanced_uncertainty______________________________ : : (9246959,)
bbox_max_latitude_________________________________ : : (9246959,)
bbox_max_longitude________________________________ : : (9246959,)
bbox_min_latitude_________________________________ : : (9246959,)
bbox_min_longitude________________________________ : : (9246959,)
code_table________________________________________ : : (9246959,)
conversion_flag___________________________________ : : (9246959,)
conversion_method_________________________________ : : (9246959,)
crs_______________________________________________ : : (9246959,)
data_policy_licence_______________________________ : : (9246959,)
date_time_________________________________________ : : 

array([    0,     1,     2, ..., 49586, 49587, 49588])

True

In [29]:
with eua.CDMDataset(
    "/mnt/users/scratch/leo/scratch/converted_v11/long/0-20001-0-11035_CEUAS_merged_v1.nc"
) as file:
    display(file)
    display(file["recordindices"])
    display(file["observations_table"])
    df = file.to_dataframe(
        groups=["observations_table"],
        variables=["z_coordinate", "z_coordinate_type", "date_time"],
    )
# df["conv_report_timestamp"] = seconds_to_datetime(df.date_time.values)
print('0-20400-0-5023_CEUAS_merged_v1.nc')
df

File: <HDF5 file "0-20001-0-11035_CEUAS_merged_v1.nc" (mode r)>
Filesize: 4632.25 MB
Filename: /mnt/users/scratch/leo/scratch/converted_v11/long/0-20001-0-11035_CEUAS_merged_v1.nc
(G)roups/(V)ariables: 

 - G | advanced_homogenisation______________________ : : 11
 - G | crs__________________________________________ : : 5
 - G | era5fb_______________________________________ : : 72
 - G | header_table_________________________________ : : 56
 - G | observations_table___________________________ : : 51
 - G | observed_variable____________________________ : : 10
 - G | recordindices________________________________ : : 13
 - G | sensor_configuration_________________________ : : 13
 - G | source_configuration_________________________ : : 3
 - G | station_configuration________________________ : : 48
 - G | station_configuration_codes__________________ : : 8
 - G | station_type_________________________________ : : 5
 - G | units________________________________________ : : 7
 - G | z_coordinate_t

recordindices:

0_________________________________________________ : : (121852,)
106_______________________________________________ : : (121852,)
107_______________________________________________ : : (121852,)
117_______________________________________________ : : (121852,)
126_______________________________________________ : : (121852,)
137_______________________________________________ : : (121852,)
138_______________________________________________ : : (121852,)
139_______________________________________________ : : (121852,)
140_______________________________________________ : : (121852,)
34________________________________________________ : : (121852,)
39________________________________________________ : : (121852,)
index_____________________________________________ : : (121852,)
recordtimestamp___________________________________ : : (121851,)

observations_table:

adjustment_id_____________________________________ : : (129269984,)
advanced_assimilation_feedback____________________ : : (129269984,)
advanced_homogenisation___________________________ : : (129269984,)
advanced_qc_______________________________________ : : (129269984,)
advanced_uncertainty______________________________ : : (129269984,)
bbox_max_latitude_________________________________ : : (129269984,)
bbox_max_longitude________________________________ : : (129269984,)
bbox_min_latitude_________________________________ : : (129269984,)
bbox_min_longitude________________________________ : : (129269984,)
code_table________________________________________ : : (129269984,)
conversion_flag___________________________________ : : (129269984,)
conversion_method_________________________________ : : (129269984,)
crs_______________________________________________ : : (129269984,)
data_policy_licence_______________________________ : : (129269984,)
date_time__________________

0-20400-0-5023_CEUAS_merged_v1.nc


,z_coordinate,z_coordinate_type,date_time
0,NaN,2,1949-04-08 03:00:00
1,NaN,2,1949-04-08 03:00:00
2,NaN,2,1949-04-12 03:00:00
3,NaN,2,1949-04-12 03:00:00
4,NaN,2,1949-04-13 03:00:00
...,...,...,...
129269979,99440.0,1,2022-12-31 11:30:23
129269980,99470.0,1,2022-12-31 11:30:23
129269981,99530.0,1,2022-12-31 11:30:23
129269982,99540.0,1,2022-12-31 11:30:23


In [21]:
z_coordinate_t = np.array([1000,750,500,800,10])
checksorting = (lambda zc: np.all(zc[:-1] >= zc[1:]))
checksorting(z_coordinate_t)

False

In [32]:
iz = iter(z_coordinate_t, 10)

TypeError: iter(v, w): v must be callable

In [31]:
next(iz)

StopIteration: 

In [18]:
z_coordinate_t[1:]

array([750, 500, 300,  10])

In [19]:
z_coordinate_t[:-1] < z_coordinate_t[1:]

array([False, False, False, False])

## H5PY
### File Access

In [40]:
with h5py.File(
    "/mnt/users/scratch/leo/scratch/converted_v11/long/0-20300-0-99010_CEUAS_merged_v1.nc",
    "r",
) as file:
    display(file['station_configuration'].keys())

<KeysViewHDF5 ['alternative_name', 'bbox_max_latitude', 'bbox_max_longitude', 'bbox_min_latitude', 'bbox_min_longitude', 'city', 'comment', 'contact', 'end_date', 'index', 'latitude', 'local_gravity', 'longitude', 'measuring_system_id', 'measuring_system_model', 'metadata_contact', 'metadata_contact_role', 'observed_variables', 'observing_frequency', 'operating_institute', 'operating_territory', 'optional_data', 'platform_sub_type', 'platform_type', 'primary_id', 'primary_id_scheme', 'record_number', 'reporting_time', 'role', 'secondary_id', 'secondary_id_scheme', 'start_date', 'station_abbreviation', 'station_automation', 'station_crs', 'station_name', 'station_type', 'string1', 'string12', 'string15', 'string198', 'string20', 'string3', 'string34', 'string4', 'string8', 'telecommunication_method']>

In [14]:
input_str = '0401820161013230000'
# file_dir = glob.glob('/mnt/users/scratch/leo/scratch/converted_v11/long/*'+str(input_str[:5])+'*_CEUAS_merged_v1.nc')[0]
file_dir = glob.glob('/mnt/users/scratch/leo/scratch/converted_v9/*'+str(input_str[:5])+'*_CEUAS_merged_v1.nc')[0]

date = str(input_str[5:9]) + '-' + str(input_str[9:11]) + '-' + str(input_str[11:13]) + ' ' + str(input_str[13:15]) + ':' + str(input_str[15:17]) 

output = {}
with h5py.File(
    file_dir,
    "r",
) as file:
    dt_from = datetime_to_seconds(np.datetime64(date))
    dt_to = dt_from + 12*60*60
    rts = file["recordindices"]["recordtimestamp"][:]
    idx = np.where(np.logical_and((rts >= dt_from), (rts <= dt_to)))[0]
    idx_d = file["recordindices"]["126"][idx]

    output['temperature'] = file["observations_table"]["observation_value"][idx_d[0]:idx_d[-1]]
    output['pressure'] = file["observations_table"]["z_coordinate"][idx_d[0]:idx_d[-1]]
    output['date_time'] = file["observations_table"]["date_time"][idx_d[0]:idx_d[-1]]
    output['sensor_id'] = file["observations_table"]["sensor_id"][idx_d[0]:idx_d[-1]]
    output['fg_depar'] = file["era5fb"]["fg_depar@body"][idx_d[0]:idx_d[-1]]
    output['an_depar'] = file["era5fb"]["an_depar@body"][idx_d[0]:idx_d[-1]]
    output['RAOBCORE_bias_estimate'] = file["advanced_homogenisation"]["RAOBCORE_bias_estimate"][idx_d[0]:idx_d[-1]]
    output['RASE_bias_estimate'] = file["advanced_homogenisation"]["RASE_bias_estimate"][idx_d[0]:idx_d[-1]]
    output['RICH_bias_estimate'] = file["advanced_homogenisation"]["RICH_bias_estimate"][idx_d[0]:idx_d[-1]]
    output['RISE_bias_estimate'] = file["advanced_homogenisation"]["RISE_bias_estimate"][idx_d[0]:idx_d[-1]]

    
df = pd.DataFrame.from_dict(output)
df

In [15]:
output

{'temperature': array([275.6, 279. , 283.4], dtype=float32),
 'pressure': array([ 86800.,  92500., 100000.], dtype=float32),
 'date_time': array([3685388400, 3685388400, 3685388400]),
 'sensor_id': array([[b'n'],
        [b'n'],
        [b'n']], dtype='|S1'),
 'fg_depar': array([ 0.7011206 , -0.02587065,  0.66992944], dtype=float32),
 'an_depar': array([ 0.29189965, -0.21811682,  0.5476524 ], dtype=float32),
 'RAOBCORE_bias_estimate': array([nan, -0., -0.], dtype=float32),
 'RASE_bias_estimate': array([nan, -0., -0.], dtype=float32),
 'RICH_bias_estimate': array([        nan, -0.09797987, -6.3157864 ], dtype=float32),
 'RISE_bias_estimate': array([        nan, -0.09797987, -6.3157864 ], dtype=float32)}

In [10]:
df = pd.DataFrame.from_dict(output)
df

ValueError: Per-column arrays must each be 1-dimensional

In [52]:
df_notnan = df[~np.isnan(df.latd)]

In [53]:
df_notnan

,latd,dt,obsv


In [6]:
file_dir = glob.glob('/users/staff/uvoggenberger/scratch/converted_v11/trajectory_files_20230502/*04018*')[0]
output = {}
with h5py.File(
    file_dir,
    "r",
) as file:
    output['latd'] = file["advanced_homogenisation"]["latitude_displacement"][:] #[idx_d[0]:idx_d[-1]]
    output['lond'] = file["advanced_homogenisation"]["longitude_displacement"][:] #[idx_d[0]:idx_d[-1]]
    output['tsl'] = file["advanced_homogenisation"]['time_since_launch'][:] #[idx_d[0]:idx_d[-1]]
    output['tt'] = file["advanced_homogenisation"]['true_time'][:] #[idx_d[0]:idx_d[-1]]
df_only = pd.DataFrame.from_dict(output)
df_only


,latd,lond,tsl,tt
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
184716884,NaN,NaN,NaN,NaN
184716885,NaN,NaN,NaN,NaN
184716886,NaN,NaN,NaN,NaN
184716887,NaN,NaN,NaN,NaN


In [7]:
df_only[~np.isnan(df_only.tsl)]

,latd,lond,tsl,tt
92173532,0.180172,-0.179681,908.756208,1.451665e+09
92173533,0.150439,-0.150822,780.267377,1.451665e+09
92173534,0.126596,-0.127699,660.122743,1.451665e+09
92173535,0.105028,-0.106799,547.238120,1.451665e+09
92173536,0.083630,-0.086079,440.509153,1.451664e+09
...,...,...,...,...
110398748,0.000114,-0.000333,4.069281,3.881430e+09
110398749,0.000071,-0.000208,2.658360,3.881430e+09
110398750,0.000036,-0.000105,1.406633,3.881430e+09
110398751,0.000019,-0.000058,0.781316,3.881430e+09


In [35]:
gfile_dir

'/users/staff/uvoggenberger/scratch/converted_v11/trajectory_files_20230426/trajectory_0-20001-0-11035_CEUAS_merged_v1.nc'

In [24]:
input_str = '0401820161013230000'
# input_str = '1103520161013230000'
file_dir = glob.glob('/mnt/users/scratch/leo/scratch/converted_v11/long/*'+str(input_str[:5])+'*_CEUAS_merged_v1.nc')[0]
# file_dir = glob.glob('/mnt/users/scratch/leo/scratch/converted_v9/*'+str(input_str[:5])+'*_CEUAS_merged_v1.nc')[0]

date = str(input_str[5:9]) + '-' + str(input_str[9:11]) + '-' + str(input_str[11:13]) + ' ' + str(input_str[13:15]) + ':' + str(input_str[15:17]) 

output = {}
with h5py.File(
    file_dir,
    "r",
) as file:
    i = 2000
    # display(file['advanced_homogenisation'].keys())
    # for i in range(1990,2020,1):
    dt_from = datetime_to_seconds(np.datetime64(str(i)+"-01-01"))
    dt_to = datetime_to_seconds(np.datetime64(str(i)+"-01-02"))
    rts = file["recordindices"]["recordtimestamp"][:]
    idx = np.where(np.logical_and((rts >= dt_from), (rts <= dt_to)))[0]
    idx_d = file["recordindices"]["126"][idx]
gfile = glob.glob('/users/staff/uvoggenberger/scratch/converted_v11/trajectory_files_20230426/*11035*')[0]
with h5py.File(
gfile,
"r",
) as gfile:
    test = gfile["advanced_homogenisation"]["longitude_displacement"][idx_d[0]:idx_d[-1]]
    print(len(test))
    print(len(test[~np.isnan(test)]))

65
0


In [12]:
dt_from

3755289600

In [49]:
idx

array([69290, 69291])

In [13]:
output

{'temperature': array([196.24, 196.25, 196.23, ..., 275.04, 274.8 , 274.89], dtype=float32),
 'pressure': array([  1680.,   1690.,   1700., ..., 100430., 100490., 100600.],
       dtype=float32),
 'date_time': array([3755329405, 3755329405, 3755329405, ..., 3786692334, 3786692334,
        3786692334]),
 'sensor_id': array([[b'1', b'4', b'2', b''],
        [b'1', b'4', b'2', b''],
        [b'1', b'4', b'2', b''],
        ...,
        [b'1', b'4', b'2', b''],
        [b'1', b'4', b'2', b''],
        [b'1', b'4', b'2', b'']], dtype='|S1'),
 'fg_depar': array([ 2.2318075 ,  2.2000031 ,  2.1381989 , ..., -0.93545824,
        -1.1558708 ,         nan], dtype=float32),
 'an_depar': array([ 1.3021971 ,  1.2760303 ,  1.2198635 , ..., -0.65341526,
        -0.8557439 ,         nan], dtype=float32),
 'RAOBCORE_bias_estimate': array([nan, nan, nan, ..., nan, nan, nan], dtype=float32),
 'RASE_bias_estimate': array([nan, nan, nan, ..., nan, nan, nan], dtype=float32),
 'RICH_bias_estimate': array([nan

In [9]:
pd.DataFrame.from_dict(output)

ValueError: Per-column arrays must each be 1-dimensional

In [36]:
input_str = '0401820161013230000'
file = glob.glob('/mnt/users/scratch/leo/scratch/converted_v11/long/*'+str(input_str[:5])+'*_CEUAS_merged_v1.nc')[0]
date = str(input_str[5:9]) + '-' + str(input_str[9:11]) + '-' + str(input_str[11:13]) + ' ' + str(input_str[13:15]) + ':' + str(input_str[15:17]) 

output = {}
with h5py.File(
    file,
    "r",
) as file:
    display(file.keys())
    display(file['observations_table'].keys())
    display(file['era5fb'].keys())
    display(file['advanced_homogenisation'].keys())

<KeysViewHDF5 ['advanced_homogenisation', 'crs', 'era5fb', 'header_table', 'observations_table', 'observed_variable', 'recordindices', 'sensor_configuration', 'source_configuration', 'station_configuration', 'station_configuration_codes', 'station_type', 'units', 'z_coordinate_type']>

<KeysViewHDF5 ['adjustment_id', 'advanced_assimilation_feedback', 'advanced_homogenisation', 'advanced_qc', 'advanced_uncertainty', 'bbox_max_latitude', 'bbox_max_longitude', 'bbox_min_latitude', 'bbox_min_longitude', 'code_table', 'conversion_flag', 'conversion_method', 'crs', 'data_policy_licence', 'date_time', 'date_time_meaning', 'exposure_of_sensor', 'index', 'latitude', 'location_method', 'location_precision', 'longitude', 'numerical_precision', 'observation_duration', 'observation_height_above_station_surface', 'observation_id', 'observation_value', 'observed_variable', 'original_code_table', 'original_precision', 'original_units', 'original_value', 'processing_level', 'quality_flag', 'report_id', 'secondary_value', 'secondary_variable', 'sensor_automation_status', 'sensor_id', 'source_id', 'spatial_representativeness', 'station_elevation', 'string21', 'string4', 'string9', 'traceability', 'units', 'value_significance', 'z_coordinate', 'z_coordinate_method', 'z_coordinate_type']

<KeysViewHDF5 ['albedo@modsurf', 'an_depar@body', 'an_depar@surfbody_feedback', 'an_sens_obs@body', 'andate', 'antime', 'biascorr@body', 'biascorr_fg@body', 'bufrtype@hdr', 'class', 'codetype@hdr', 'collection_identifier@conv', 'date@hdr', 'datum_anflag@body', 'datum_event1@body', 'datum_rdbflag@body', 'datum_sfc_event@surfbody_feedback', 'datum_status@body', 'datum_status@surfbody_feedback', 'eda_spread@errstat', 'entryno@body', 'expver', 'fg_depar@body', 'fg_depar@surfbody_feedback', 'fg_error@errstat', 'final_obs_error@errstat', 'groupid@hdr', 'index', 'lat@hdr', 'lon@hdr', 'lsm@modsurf', 'lsm@surfbody_feedback', 'numtsl@desc', 'obs_error@errstat', 'obstype@hdr', 'obsvalue@body', 'orography@modsurf', 'ppcode@conv_body', 'qc_pge@body', 'report_event1@hdr', 'report_rdbflag@hdr', 'report_status@hdr', 'reportype', 'seaice@modsurf', 'sensor@hdr', 'seqno@hdr', 'snow_density@surfbody_feedback', 'snow_depth@modsurf', 'snow_depth@surfbody_feedback', 'sonde_type@conv', 'source@hdr', 'source_i

<KeysViewHDF5 ['RAOBCORE_bias_estimate', 'RASE_bias_estimate', 'RICH_bias_estimate', 'RISE_bias_estimate', 'humidity_bias_estimate', 'index', 'latitude_displacement', 'longitude_displacement', 'time_since_launch', 'true_time', 'wind_bias_estimate']>

In [29]:
date = str(input_str[5:9]) + '-' + str(input_str[9:11]) + '-' + str(input_str[11:13]) + ' ' + str(input_str[13:15]) + ':' + str(input_str[15:17])

In [30]:
date

'2016-10-13 23:00'

### Dataframe concatination with h5py

In [ ]:
conv_file = glob.glob(
    "/mnt/users/scratch/leo/scratch/converted_v11/long//0-20001-0-11035_CEUAS_merged_v1.nc"
)[0]
dt_from = datetime_to_seconds(np.datetime64("1970-01-01"))
dt_to = datetime_to_seconds(np.datetime64("1970-01-02"))

df_dict = {}
h_df_dict = {}

with h5py.File(conv_file, "r") as file:
    rts = file["recordindices"]["recordtimestamp"][:]
    idx = np.where(np.logical_and((rts >= dt_from), (rts <= dt_to)))[0]
    plevs = [
        1000,
        2000,
        3000,
        5000,
        7000,
        10000,
        15000,
        20000,
        25000,
        30000,
        40000,
        50000,
        70000,
        85000,
        92500,
        100000,
    ]

    idx_d = {}
    var_d = {
        "air_temperature": "126",
        "relative_humidty": "138",
        "geopotential": "117",
        "eastward_wind_speed": "139",
        "northward_wind_speed": "140",
        "dew_point": "137",
        "specific_humidity": "39",
    }
    for i in var_d:
        idx_d[i] = file["recordindices"][var_d[i]][idx]

    masks = {}
    for i in idx_d:
        masks[i] = file["observations_table"]["z_coordinate"][
            idx_d[i][0] : idx_d[i][-1]
        ]
        masks[i] = np.isin(masks[i], plevs)
        # masks[i] = np.isfinite(masks[i])

    mask = masks["air_temperature"]
    t_idx = idx_d["air_temperature"]
    df_dict["z_coordinate"] = list(
        file["observations_table"]["z_coordinate"][t_idx[0] : t_idx[-1]][mask]
    )
    df_dict["date_time"] = seconds_to_datetime(
        list(file["observations_table"]["date_time"][t_idx[0] : t_idx[-1]][mask])
    )
    df_dict["latitude"] = list(
        file["observations_table"]["latitude"][t_idx[0] : t_idx[-1]][mask]
    )
    df_dict["longitude"] = list(
        file["observations_table"]["longitude"][t_idx[0] : t_idx[-1]][mask]
    )
    repid = np.asarray(
        file["observations_table"]["report_id"][t_idx[0] : t_idx[-1]][mask]
    )
    df_dict["report_id"] = list(
        repid.view("|S{}".format(repid.shape[1])).flatten().astype(str)
    )
    # df_dict['RASE_bias_estimate'] = list(file['advanced_homogenisation']['RASE_bias_estimate'][t_idx[0]:t_idx[-1]][mask])
    # df_dict['latitude_displacement'] = list(file['advanced_homogenisation']['latitude_displacement'][t_idx[0]:t_idx[-1]][mask])
    # df_dict['longitude_displacement'] = list(file['advanced_homogenisation']['longitude_displacement'][t_idx[0]:t_idx[-1]][mask])
    # df_dict['time_since_launch'] = list(file['advanced_homogenisation']['time_since_launch'][t_idx[0]:t_idx[-1]][mask])
    df_dict["air_temperature"] = list(
        file["observations_table"]["observation_value"][t_idx[0] : t_idx[-1]][mask]
    )

    df_dict_d = {}
    for i in masks:
        if i != "air_temperature":
            df_dict_d[i] = {}
    for i in masks:
        if i != "air_temperature":
            df_dict_d[i]["z_coordinate"] = list(
                file["observations_table"]["z_coordinate"][idx_d[i][0] : idx_d[i][-1]][
                    masks[i]
                ]
            )
            df_dict_d[i]["date_time"] = seconds_to_datetime(
                list(
                    file["observations_table"]["date_time"][idx_d[i][0] : idx_d[i][-1]][
                        masks[i]
                    ]
                )
            )
            df_dict_d[i][i] = list(
                file["observations_table"]["observation_value"][
                    idx_d[i][0] : idx_d[i][-1]
                ][masks[i]]
            )

    # df_dict['date_time'] = seconds_to_datetime(df_dict['date_time'])
    df = pd.DataFrame.from_dict(df_dict)

    # h_df_dict['date_time'] = seconds_to_datetime(h_df_dict['date_time'])
    h_df = pd.DataFrame.from_dict(h_df_dict)

    # put dfs together:
    for i in df_dict_d:
        if i != "air_temperature":
            df = df.merge(
                pd.DataFrame.from_dict(df_dict_d[i]),
                how="left",
                on=["date_time", "z_coordinate"],
            )
df